## **<h3 align="center"> Computational Intelligence for Optimization</h3>**
# **<h3 align="center">Testing Crossover Operators</h3>**
**Group members:**<br>
Alexandra Pinto - 20211599@novaims.unl.pt - 20211599<br>
Julia Karpienia  - 20240514@novaims.unl.pt - 20240514<br>
Steven Carlson - 20240554@novaims.unl.pt - 20240554 <br>
Tim Straub - 20240505@novaims.unl.pt - 20240505

<a id = "toc"></a>

## Table of Contents
1. [Introduction](#intro)
2. [Import & Setup](#import_setup)
3. [Generate Initial Population](#gen_initial_pop)
4. [Define General Crossover Testing Function](#testing_crossover_function)
5. [Run Crossover Tests](#run_crossovers_test)
   - 5.1. [Swap Whole Position Crossover](#swap_whole_positision)
   - 5.2. [Extreme Player Swap Crossover](#extreme_player_swap)
   - 5.3. [Swap Teams Crossover](#swap_teams)
6. [Conclusion](#conclusion)

#  1. Introduction <a class="anchor" id="intro"></a> 
[Back to ToC](#toc)<br>

This notebook is used to **test and understand how each crossover operator behaves**. We run multiple trials to evaluate if the offspring generated are valid, different from their parents, and follow the intended logic.

We also added **print statements inside some of the crossover functions** to help visualize the process and debug. These prints will be **commented out later** to avoid excessive output during the final Grid Search and evaluation phases.


# 2. Import & Setup <a class="anchor" id="import_setup"></a>

[Back to ToC](#toc)<br>


In [ ]:
# From the Operators folder, import all the necessary modules
# To create a new population, we need to import the following:
from Operators.population import *
# To call the crossovers, we need to import the following:
from Operators.crossovers import *
from collections import Counter


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



# 3. Generate Initial Population <a class="anchor" id="gen_initial_pop"></a>

[Back to ToC](#toc)<br>

In this section, we use the functions `load_players_from_csv` and `generate_population` from `population.py` to create the initial population. This population will serve as the basis for testing the crossover operators in the following sections.

In [33]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

Let's generate 10 leagues.

In [ ]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    # With the league object, we can access its attributes and methods, in this case we will access to the standard deviation of the average skills
    print(f"Standard Deviation of Avg Skills: {league.get_skill_std_dev():.2f}")


--- League 1 ---
Chris Thompson (GK) Skill: 80 Salary: €80.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Owen Parker (DEF) Skill: 88 Salary: €100.0M
Bentley Rivera (MID) Skill: 88 Salary: €100.0M
Austin Torres (MID) Skill: 82 Salary: €80.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M
Adrian Collins (FWD) Skill: 85 Salary: €90.0M

Alex Carter (GK) Skill: 85 Salary: €90.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Spencer Ward (MID) Skill: 84 Salary: €85.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Zachary Nelson (FWD) Skill: 86 Salary: €92.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M

Blake Henderson (GK) Skill: 87 Salary: €95.0M
Ethan Howard (DEF) Skill: 80 Salary: €70.0M
Brayden Hughes (DEF) Skill: 87 Salary: €100.0M
Ashton Phillips (MID) Skill: 90 Salary: €110.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Tyler Jenkins (FWD) Skill: 80 Salary: €70.0M
Sebastian Perry (FWD) Skill: 95 Salary: €150.0M

Ryan Mitchell (GK) Skill:

# 4. Define General Crossover Testing Function <a class="anchor" id="testing_crossover_function"></a>

[Back to ToC](#toc)<br>
Let's run a series of randomized trials using the `test_crossover` function to assess:

- Whether the crossover produces valid children.
- How often the operation has no effect (no-op).
- If any invalid outputs are generated, taking into account the constraints.


In [ ]:
def test_crossover(
    population: list[League],
    crossover_fn,
    trials: int = 100,
    *crossover_args
):
    """
    General tester for any 2-parent → 2-child crossover function.

    Parameters:
    -----------
    population    : list[League]
        List of parent leagues to sample from.
    crossover_fn  : callable
        Crossover function with signature:
            child1, child2 = crossover_fn(parent1, parent2, *crossover_args)
    trials        : int
        Number of random parent-pairs to test.
    crossover_args: any
        Additional positional arguments to pass into crossover_fn.

    Behavior:
    ---------
    - For each trial:
      1. Randomly pick two distinct parents p1, p2.
      2. Deep‐copy them as orig1, orig2.
      3. Call: child1, child2 = crossover_fn(p1, p2, *crossover_args)
      4. Classify outcome:
         • “invalid” if either child is None
         • “no-op”   if both children equal their respective originals
         • “valid”   otherwise
    - Prints a summary table of counts and percentages.
    """
    stats = Counter()
    
    for _ in range(trials):
        p1, p2 = random.sample(population, 2)
        orig1, orig2 = deepcopy(p1), deepcopy(p2)
        
        # Run crossover
        child1, child2 = crossover_fn(p1, p2, *crossover_args)
        
        # Classify
        if child1 is None or child2 is None:
            stats['invalid'] += 1
        else:
            if str(child1) == str(orig1) and str(child2) == str(orig2):
                stats['no-op'] += 1
            else:
                stats['valid'] += 1

    # Report
    total = sum(stats.values())
    print(f"\nCrossover tests for {crossover_fn.__name__} over {trials} trials:")
    for category in ['valid', 'no-op', 'invalid']:
        count = stats.get(category, 0)
        pct = (count / total * 100) if total else 0
        print(f"  {category:7s}: {count:3d}  ({pct:5.1f}%)")


# 5. Run Crossover Tests <a class="anchor" id="run_crossovers_test"></a>
[Back to ToC](#toc)<br>

## 5.1. Swap Whole Position Crossover <a class="anchor" id="swap_whole_position"></a>

[Back to 5. Run Crossover Tests](#run_crossovers_test)<br>
This crossover selects a single position (GK, DEF, MID, or FWD) and swaps all players of that position between corresponding teams from two different leagues. It helps mix player blocks while preserving team structure.

**Steps:**

1. Randomly pick one position (e.g., 'MID').

2. For each team in the same index from both leagues, swap players at that position.

3. Skip the swap for a team pair if either lacks players in that position.

**Example:**
Suppose we have two leagues:

- League A – Team A1: MID = [A4, A5]

- League B – Team B1: MID = [B4, B5]

After crossover (if 'MID' is selected):
- Team A1 (from League A) gets [B4, B5]

- Team B1 (from League B) gets [A4, A5]

This allows injecting new skill dynamics into teams while maintaining positional balance.

In [37]:
test_crossover(population, crossover_swap_whole_position, trials=200)

Swapping all players at position: MID
Swapping all players at position: DEF
Swapping all players at position: MID
Swapping all players at position: MID
Swapping all players at position: MID
Swapping all players at position: MID
Swapping all players at position: MID
Swapping all players at position: GK
Swapping all players at position: GK
Swapping all players at position: FWD
Swapping all players at position: GK
Swapping all players at position: GK
Swapping all players at position: MID
Swapping all players at position: DEF
Swapping all players at position: DEF
Swapping all players at position: MID
Swapping all players at position: FWD
Swapping all players at position: MID
Swapping all players at position: GK
Swapping all players at position: DEF
Swapping all players at position: DEF
Swapping all players at position: DEF
Swapping all players at position: FWD
Swapping all players at position: GK
Swapping all players at position: GK
Swapping all players at position: GK
Swapping all players


## 5.2. Extreme Player Swap Crossover <a class="anchor" id="extreme_player_swap"></a>

[Back to 5. Run Crossover Tests](#run_crossovers_test)<br>
This crossover identifies the most “extreme” player , the one whose skill deviates most from the average of their position in the team, and swaps that player between corresponding teams in the two different leagues. The goal is to introduce diversity by targeting outlier players.

**Steps:**

1. Randomly pick one position (e.g., 'DEF').

2. For each team pair in the same index across both leagues:

    2.1. Compute the average skill of the players in the selected position.

    2.2. Find the player with the largest absolute deviation from this average.

    2.3. Swap these two "extreme" players between teams.

3. Skip the swap if a team lacks players in the selected position.

**Example:**

- League A – Team A1 (DEF): Players = [60, 85] → Average = 72.5

Both players deviate equally (12.5), so one is selected randomly. Say we pick A1 = 60.

- League B – Team B1 (DEF): Players = [70, 95] → Average = 82.5

Deviation: B1 = 70 → 12.5, B2 = 95 → also 12.5 → Randomly select one. Say we pick B2 = 95.

After crossover:

- Team A1 gets 95 (from B)
- Team B1 gets 60 (from A)

This operator enhances genetic diversity by swapping players with outlier characteristics.



In [38]:
test_crossover(population, crossover_swap_extreme_player, trials=200)


Crossover tests for crossover_swap_extreme_player over 200 trials:
  valid  : 135  ( 67.5%)
  no-op  :  35  ( 17.5%)
  invalid:  30  ( 15.0%)



## 5.3. Swap Teams Crossover <a class="anchor" id="swap_teams"></a>
[Back to 5. Run Crossover Tests](#run_crossovers_test)<br>

This crossover builds new leagues by taking entire teams from each parent league, using fixed team indices. It ensures player uniqueness across the resulting league and maintains valid team structures.

**Steps:**

1. Decide which indices will be inherited from each parent. For example:

    Child 1 takes teams [0, 2, 4] from League A and [1, 3] from League B.

    Child 2 takes teams [0, 2, 4] from League B and [1, 3] from League A.

2. Assemble the teams into two new leagues.

3. Ensure no player appears in both teams (i.e., remove duplicates if necessary).

**Example:**

- League A Teams: [A1, A2, A3, A4, A5]

- League B Teams: [B1, B2, B3, B4, B5]

After crossover:

- Child League 1: [A1, A3, A5] + [B2, B4]

- Child League 2: [B1, B3, B5] + [A2, A4]

This crossover preserves team-level strategies and introduces high-level structural diversity.

In [ ]:
def crossover_team(parent1, parent2):
    """
    Performs a deterministic crossover using a fixed team selection pattern from each parent.
    Ensures that no player appears more than once in the resulting child leagues.

    Pattern:
    Child 1: Teams 1, 3, 5 from parent1; Teams 2, 4 from parent2
    Child 2: Teams 1, 3, 5 from parent2; Teams 2, 4 from parent1

    Parameters:
    -----------
    parent1 : League
        First parent league.
    parent2 : League
        Second parent league.

    Returns:
    --------
    tuple : (League, League)
        Two new child leagues built from the preset team pattern. If duplicates are found,
        it raises an error.
    """

    def build_child(pattern, source1, source2):
        teams = []
        for idx in pattern:
            if idx[0] == 1:
                teams.append(deepcopy(source1.teams[idx[1]]))
            else:
                teams.append(deepcopy(source2.teams[idx[1]]))
        return League(teams)

    # Define the selection pattern: (source_league, team_index)
    pattern_child1 = [(1, 0), (2, 1), (1, 2), (2, 3), (1, 4)]
    pattern_child2 = [(2, 0), (1, 1), (2, 2), (1, 3), (2, 4)]

    child1 = build_child(pattern_child1, parent1, parent2)
    child2 = build_child(pattern_child2, parent1, parent2)

    # Validate for duplicate players
    def validate_unique_players(league, child_name):
        player_names = set()
        for i, team in enumerate(league.teams):
            for player in team.players:
                if player.name in player_names:
                    raise ValueError(
                        #f"{child_name} is invalid: Duplicate player {player.name} in team {i+1}."
                    )
                player_names.add(player.name)

    try:
        validate_unique_players(child1, "Child 1")
        validate_unique_players(child2, "Child 2")
    except ValueError as e:
        #print("Validation failed:", e)
        return None, None

    #print("Preset team mix crossover successful.")
    return child1, child2


In [42]:
test_crossover(population, crossover_team, trials=200)

ValueError: Player Julian Scott is already in another team.

# 6. Conclusion <a class="anchor" id="conclusion"></a>
[Back to ToC](#toc)<br>

After running 200 trials for each crossover strategy, we obtained the following results:

| **Crossover Name**  | **Function Name**               | **Valid (%)** | **No-op (%)** | **Invalid (%)** | **Insights**                                                                     |
| ------------------- | ------------------------------- | ------------- | ------------- | --------------- | -------------------------------------------------------------------------------- |
| Swap Whole Position | `crossover_swap_whole_position` | 100.0%        | 0.0%          | 0.0%            | Consistently produces valid offspring; robust and reliable.                      |
| Extreme Player Swap | `crossover_swap_extreme_player` | 67.5%         | 17.5%         | 15.0%           | Encourages diversity via extreme players but occasionally breaks constraints.    |
| Swap Teams          | `crossover_team`                | 0.0%          | 0.0%          | 100.0%          | Always invalid due to duplicate players; **excluded** from final implementation. |


Note:  Based on these results, crossover_team was not included in crossovers.py due to its consistent production of invalid offspring.